In [2]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.stats import geom

# question 1

In [1]:
class Rejection_sampler:
    def __init__(self, p, I):  # Default value for m
        self.p = np.array(p)
        self.I = I
        self.N = len(p)
        self.g = self.pdf_bernoulli(p)  # Bernoulli probability function
        self.f = self.pdf_cb(p, I)  # Conditional probability function
        self.M = self.compute_M(self)  # Compute M
        self._sample = None
        self._acceptances = None
    
    @staticmethod
    def compute_M(self):
        """Computes M based on given m and probability ratios"""
        res = 1
        for seq in self.generate_sequences(self.N, self.I):  # Corrected N and I
            pr = self.f(seq) / self.g(seq)  # Fix missing self.
            if pr > res:
                res = pr
        return res

    def pdf_cb(self, p, I):
        """Computes a conditional probability function"""
        g = self.pdf_bernoulli(p)
        proba = 0
        for x in self.generate_sequences(len(p), I):
            proba += g(x)

        def f(x):
            return g(x) / proba if np.sum(x) == I else 0  # Use np.sum(x)

        return f

    @staticmethod
    def generate_sequences(N, I):
        """Generates all binary sequences of length N with I ones"""
        positions = itertools.combinations(range(N), I)
        sequences = []
        for pos in positions:
            seq = np.zeros(N, dtype=int)  # Start with all zeros
            seq[list(pos)] = 1  # Set the specified positions to 1
            sequences.append(seq)
        return np.array(sequences)

    @staticmethod
    def pdf_bernoulli(p):
        """Creates a Bernoulli probability function"""
        def g(x):
            return np.prod(np.where(x == 1, p, 1 - p))
        return g
    
    def sample(self,L=1):
        samples = []
        acceptances = []
        while len(samples) < L:
            attempt = 0
            while True:
                attempt += 1 
                X = np.array([np.random.binomial(1, p_i) for p_i in self.p])
                U = np.random.uniform(0,1)
                if U<=(self.f(X)/(self.g(X)*(self.M))):
                    samples.append(X)
                    acceptances.append(attempt)
                    break
        self._sample=np.array(samples)
        self._acceptances = np.array(acceptances)
    
    def plot_acceptance_density(self):
        """Plots the empirical density function and the theoretical Geometric(1/M) density."""
        if self._acceptances is None:
            print("No samples generated yet. Run sample() first.")
            return

        unique, counts = np.unique(self._acceptances, return_counts=True)
        empirical_density = counts / np.sum(counts)  # Normalize to get probability values

        # Theoretical geometric distribution with success probability 1/M
        k_values = np.arange(1, max(unique) + 1)  # Range of values for plotting
        theoretical_density = geom.pmf(k_values, 1 / self.M)  # Geometric PMF

        plt.figure(figsize=(8, 5))
        
        # Empirical density as a bar plot
        plt.bar(unique, empirical_density, width=0.8, color='b', alpha=0.7, edgecolor='black', label="Empirical Density")
        
        # Theoretical geometric density as a line plot
        plt.plot(k_values, theoretical_density, 'r-o', markersize=4, label=f"Theoretical Geom(1/{self.M:.2f})", linewidth=1)

        plt.xlabel("Number of Attempts Before Acceptance")
        plt.ylabel("Probability Density")
        plt.title("Empirical vs Theoretical Density of Acceptance Attempts")
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# question 2

# question 3

In [ ]:
class MCMC_sampler:
    def __init__(self,p,I,):
        self.p = np.array(p)
        self.I = I
        self.N = len(p)
        self.k = self.N*np.log(self.N)
        self._sample = None
        self._sequences = None
        self._acceptances = None
    
    def sample_one(self,k = None, conserve_sequence = False):
        if self.I ==0:
            print('erreur pas deterministe')
            return None
            
        
        res_seq = []
        if k is None:
            k=self.k
        X = np.zeros(self.N, dtype=int)
        X[:self.I] = 1
        np.random.shuffle(X)
        w = self.p/(1-self.p)
        S0 = {index : None for index in range(self.N) if X[index] == 1}
        S1 = {index : None for index in range(self.N) if X[index] == 1}
        
        for _ in range(k):
            i_0 = np.random.choice(S0.keys())
            i_1 = np.random.choice(S1.keys())
            alpha = min(1,w[i_0]/w[i_1])
            
            if np.random.rand() < alpha:
                if conserve_sequence : 
                    res_seq.append(X)
                X[i_1],X[i_0] = 0,1
                
                S0.pop(i_0)
                S1.pop(i_1)
                S0[i_1] = None
                S1[i_0] = None
        
        if conserve_sequence:
            return (X, np.array(res_seq))
        return X